# Tide Prediction Task

Start with some imports.

In [1]:
from gbdxtools import Interface
from shapely.wkt import loads
import dateutil.parser
from datetime import datetime

Create a GBDX interface.

In [2]:
gbdx = Interface()

Start with a catalog ID (this is some image of the Palm Islands in Dubai).

In [13]:
cat_id = '103001000349F800'

Get metadata record for parsing of footprint WKT and timestamp.

In [14]:
record = gbdx.catalog.get(cat_id)

Get the centroid of the footprint.

In [15]:
centroid = loads(record.get('properties').get('footprintWkt')).centroid

The latitude corresponds to `y` and longitude corresponds to `x`.

In [16]:
lat = centroid.y
lon = centroid.x
print('lat=%f, lon=%f' % (lat, lon))

lat=25.113527, lon=55.133696


Next, we get the timestamp of the image.

In [17]:
timestamp = dateutil.parser.parse(record.get('properties').get('timestamp'))

The timestamp is converted to the date time group format that the tide prediction algorithm expects (Y-m-d-H-M).

In [18]:
dtg = datetime.strftime(timestamp, '%Y-%m-%d-%H-%M')

In [19]:
print('dtg="%s"' % dtg)

dtg="2010-01-12-07-12"


At this point, we'd like to call the task with

In [21]:
print('lat="%s", lon="%s", dtg="%s"' % (str(lat), str(lon), dtg))

lat="25.1135269383", lon="55.1336961813", dtg="2010-01-12-07-12"


and receive an output like

```json
{
  "minimumTide24Hours":null,
  "maximumTide24Hours":null,
  "currentTide":null
}
```

We have currently implemented tide prediction using a modified version of bf-tideprediction (just without the flask wrapper for endpoints). The image `chambbj/hello-gbdx` v0.0.9 is available as GBDX task `hello-gbdx-chambbj`. It takes the aforementioned lat, lon, dtg all as strings.

In [259]:
# tide_task = gbdx.Task('bf-tideprediction-gbdx')
tide_task = gbdx.Task('hello-gbdx-chambbj')
tide_task.inputs.lat = str(lat)
tide_task.inputs.lon = str(lon)
tide_task.inputs.dtg = dtg

The tide prediction json file will be saved to `/mnt/work/output/json` and is persisted to `some_new_folder` within my user bucket/prefix on S3.

In [260]:
workflow = gbdx.Workflow([tide_task])
workflow.savedata(tide_task.outputs.json, location='some_new_folder')
workflow.execute()
workflow.status

{u'event': u'submitted', u'state': u'pending'}

Check periodically for status `complete`.

In [310]:
workflow.status

{u'event': u'succeeded', u'state': u'complete'}

And download the result. Yes, we could display it here, but this is good enough.

In [311]:
gbdx.s3.download('some_new_folder')

The tides.json file is now in the local directory.